In [1]:
import keras
import numpy as np
import pandas as pd
import scipy.io as sio
from tcn import TCN,tcn_full_summary
from tensorflow.keras.layers import Dense,Input,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
import time
from keras.models import Model
from tensorflow.keras.utils import to_categorical

In [2]:
#Import data
input_train=sio.loadmat('input_train60_1.3.mat')
input_real_train=sio.loadmat('input_real_train60_1.3.mat')
output_train=sio.loadmat('output_train60_1.3.mat')
input_train=input_train['input_train']
input_real_train=input_real_train['input_real_train']
output_train=output_train['output_train']
input_test=sio.loadmat('input_test60_1.3.mat')
output_test=sio.loadmat('output_test60_1.3.mat')
input_real_test=sio.loadmat('input_real_test60_1.3.mat')
input_test=input_test['input_test']
output_test=output_test['output_test']
input_real_test=input_real_test['input_real_test']
input_train=np.transpose(input_train)
input_real_train=np.transpose(input_real_train)
output_train=np.transpose(output_train)
input_test=np.transpose(input_test)
output_test=np.transpose(output_test)
input_real_test=np.transpose(input_real_test)


In [3]:
#Normalize and generate training set and test set
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler(feature_range=(0, 1))
inputn_train=scaler1.fit_transform(input_train)
scaler3 = MinMaxScaler(feature_range=(0, 1))
inputn_real_train=scaler3.fit_transform(input_real_train)
scaler2 = MinMaxScaler(feature_range=(0, 1))
outputn_train=scaler2.fit_transform(output_train)
inputn_test=scaler1.transform(input_test)
inputn_real_test=scaler3.transform(input_real_test)
outputn_test=scaler2.transform(output_test)
outputn_test.shape
XTrain1=np.zeros((12060,20,4))
XTrain2=np.zeros((12060,20,4))
XTrain3=np.zeros((12060,60,1))
YTrain=np.zeros((12060,60))

for i in range(12060):
    XTrain1[i,:,0]=inputn_train[i:i+20,0]
    XTrain1[i,:,1]=inputn_train[i:i+20,1]
    XTrain1[i,:,2]=inputn_train[i:i+20,2]
    XTrain1[i,:,3]=inputn_train[i:i+20,3]
    XTrain2[i,:,0]=inputn_train[i:i+20,0]
    XTrain2[i,:,1]=inputn_train[i:i+20,1]
    XTrain2[i,:,2]=inputn_train[i:i+20,2]
    XTrain2[i,:,3]=inputn_train[i:i+20,3]
    XTrain3[i,:]=inputn_real_train[i:i+60]
    YTrain[i,:]=outputn_train[i,:]
XTest1=np.zeros((12000,20,4))
XTest2=np.zeros((12000,20,4))
XTest3=np.zeros((12000,60,1))
YTest=np.zeros((12000,60))
for i in range(12000):
    XTest1[i,:,0]=inputn_test[i:i+20,0]
    XTest1[i,:,1]=inputn_test[i:i+20,1]
    XTest1[i,:,2]=inputn_test[i:i+20,2]
    XTest1[i,:,3]=inputn_test[i:i+20,3]
    XTest2[i,:,0]=inputn_test[i:i+20,0]
    XTest2[i,:,1]=inputn_test[i:i+20,1]
    XTest2[i,:,2]=inputn_test[i:i+20,2]
    XTest2[i,:,3]=inputn_test[i:i+20,3]
    XTest3[i,:]=inputn_real_test[i:i+60]
    YTest[i,:]=outputn_test[i,:];

In [4]:
#Establish network model
input1 = Input(batch_shape=(None,20,4), name='input1')
input2 = Input(batch_shape=(None,20,4), name='input2')
input3=Input(batch_shape=(None,60), name='input3')
batch_size, time_steps, input_dim = None, 20, 4
tcn_layer1=TCN(input_shape=(time_steps, input_dim),nb_filters=64,
    kernel_size=3,dilations=[1,2,4])
x=tcn_layer1(input1)
x=Dense(60)(x)


y=LSTM(64, input_dim=4, input_length=20, return_sequences=True)(input2)
y=LSTM(64, input_dim=4, input_length=20, return_sequences=False)(y)
y=Dense(128,activation='relu')(y)
y=Dense(60)(y)


r=keras.layers.concatenate([x,y,input3])
r = Dense(64, activation='relu')(r)


output = Dense(60, name='main_output')(r)
model = Model(inputs=[input1,input2,input3], outputs=[output])
def get_lr_metric(optimizer):  # printing the value of the learning rate
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
optimizer = keras.optimizers.Adam(lr=0.0001)
lr_metric = get_lr_metric(optimizer)
model.compile(optimizer = optimizer, loss = 'mse', metrics = [lr_metric])

In [5]:
#Training network
from keras.callbacks import ReduceLROnPlateau
import keras.backend as K
from keras.callbacks import LearningRateScheduler
 
def scheduler(epoch):
    if epoch % 200 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)
 
reduce_lr = LearningRateScheduler(scheduler)
start=time.process_time()
history=model.fit([XTrain1,XTrain2,XTrain3], YTrain, epochs=300, validation_split=0.2,callbacks=[reduce_lr])

Epoch 1/300
235/302 [======================>.......] - ETA: 0s - loss: 0.5473 - lr: 1.0000e-04

KeyboardInterrupt: 

In [ ]:
#Loss
import matplotlib.pyplot as plt
loss = history.history['loss']         
val_loss = history.history['val_loss'] 
epochs = range(1,len(loss)+1)
plt.figure()   
plt.plot(epochs,val_loss,'b',label='Vaildation loss')
plt.legend()

In [ ]:
#Test
YPred0=model.predict([XTrain1,XTrain2,XTrain3])
YPred=model.predict([XTest1,XTest2,XTest3])
YPred0=scaler2.inverse_transform(YPred0)
YPred=scaler2.inverse_transform(YPred)
YTest=scaler2.inverse_transform(YTest)

In [ ]:
mean_squared_error(YTest[10:10000,:],YPred[10:10000,:])

In [ ]:
#Save data
from scipy.io import savemat
savemat('TCN_LSTM_regression_loss.mat',{'TCN_LSTM_val_loss':val_loss})
savemat('LSTM_TCN_YPred_60_1.3.mat',{'TCN_LSTM_YPred':YPred})
savemat('LSTM_TCN_YPred0_60_1.3.mat',{'LSTM_TCN_YPred0_60_1.3.1':YPred0})